<a href="https://colab.research.google.com/github/kavyareddy01/Finetuning/blob/main/FineTuning_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
torch.__version__

'2.1.0+cu121'

In [1]:
pip install --upgrade torch torchvision

### 1.**INSTALL ALL REQUIRED PACKAGES**




In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00


PEFT:Parameterr-Efficient Transfer Learning for NLP(freezes weights and some weights are retrained and provide accurate result based on custom dataset)

LoRA:Low-Rank Adaption of Large Language Models

bitsandbytes : specifically used for quantisation(float32-int8)

---



###2.**IMPORT ALL THE REQ LIBRARIES**

In [ ]:
import os
import torch

In [ ]:
from datasets import load_dataset

In [8]:
import torch
print(torch.cuda.is_available())

False


In [6]:
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [ ]:
from peft import LoraConfig,PeftModel

In [7]:
from trl import SFTTrainer

#Incase of llama 2, the following prompt template is used for the chat models
* System Prompt(optional) to guide the model
* User Prompt(required) to give the instruction
* Model Answer(required)


#We will reformat our instruction dataset to follow Llama 2 template